In [92]:
''' heavily borrowed from https://github.com/GalvanizeDataScience/lectures/blob/Denver/text-classification/frank-burkholder/naive_bayes_sklearn.ipynb'''


' heavily borrowed from https://github.com/GalvanizeDataScience/lectures/blob/Denver/text-classification/frank-burkholder/naive_bayes_sklearn.ipynb'

In [93]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score


In [127]:
categories = ['adoptable', 'adopted']

In [128]:
path_csv = '../data/csv/giant_valid_csv.csv'
df1 = pd.read_csv(path_csv)
# df = df.dropna()
# print(df.columns)
df = df1.fillna("None")
df.drop(['photos', 'videos', 'distance', 'status_changed_at', 'published_at', 'distance', 'contact', 'organization_animal_id', 'type', 'photos'], axis = 1, inplace= True)

df = pd.get_dummies(df, columns=['status'])
df.drop("status_adoptable", axis = 1, inplace=True)
print(df.shape)
# new = old[['A', 'C', 'D']].copy()

df_content = df[["status_adopted", "description"]].copy()
# print(df_content.head())

X = df_content["description"] #data
X.to_numpy()
print(X)

y = df_content["status_adopted"] #target
y.to_numpy()
# print(y)  0 == negative,  1 == positive                                     

(722, 18)
0      Meet Ruff, Ruff is a smaller breed - probably ...
1      ______ DOB/AGE: 07/10/2019 WEIGHT (GROWN):  55...
2      Okay people, prepare yourself for this face. M...
3      Louie is 15 months old and 42 lbs. He is a Ger...
4      My name is Kenny. I&#039;m a 1 yr old, male bo...
                             ...                        
717    Dina is only about 1 1/2 yrs old.   Adoption f...
718    Meet Virgil! Virgil is a happy happy dog! He i...
719    Meet Omar, Omar will be a small/medium sized d...
720    DORA  Female SHEP MIX BRINDLE 13 2DHPP KC 2/15...
721    Meet Nova!  Now that she is done raising her p...
Name: description, Length: 722, dtype: object


array([1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1,

In [129]:
print(X[0])

Meet Ruff, Ruff is a smaller breed - probably around 10 lbs at  6-7  months old and a complete mix...


In [130]:
# eighty_percent_of_data = int(len(X) * .8)
# twenty_percent_of_data = int(len(X) * .2)

# X_train = X[:eighty_percent_of_data]
# y_train = y[:eighty_percent_of_data]
# X_test = X[:twenty_percent_of_data]
# y_test = y[:eighty_percent_of_data]
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [131]:
# print(df_content.head())
# print(X)
# print(y)
print(len(X)) #577
print(len(y)) #577

722
722


In [158]:
#TODO make my own stopwords list
import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words("english"))
new_stopwords = ['old', 'mix', 'dogs', '039', 'amp', 'sweet', 'year', 'years', 'website' , 'loves', 'adoption', 'application', 'shelter', 'rescue', 'rescued']
new_stopwords_list = stop_words.union(new_stopwords)

count_vect = CountVectorizer(lowercase=True, tokenizer=None, stop_words = new_stopwords_list,  analyzer='word', max_df=1.0, min_df=1,  max_features=None) 
                             

count_vect.fit(X)

target_names = y

In [159]:
print(count_vect)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None,
                stop_words={'039', 'a', 'about', 'above', 'adoption', 'after',
                            'again', 'against', 'ain', 'all', 'am', 'amp', 'an',
                            'and', 'any', 'application', 'are', 'aren',
                            "aren't", 'as', 'at', 'be', 'because', 'been',
                            'before', 'being', 'below', 'between', 'both',
                            'but', ...},
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)


In [160]:

X_counts = count_vect.transform(X)
print("The type of X_counts is {0}.".format(type(X_counts)))
print("The X matrix has {0} rows (documents) and {1} columns (words).".format(
        X_counts.shape[0], X_counts.shape[1]))

The type of X_counts is <class 'scipy.sparse.csr.csr_matrix'>.
The X matrix has 722 rows (documents) and 1806 columns (words).


In [161]:
tfidf_transformer = TfidfTransformer(use_idf=True)
tfidf_transformer.fit(X_counts)
X_tfidf = tfidf_transformer.transform(X_counts)

In [162]:
# y.values

In [163]:
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y.values, test_size=0.2, random_state=42)

In [164]:
nb_model = MultinomialNB(alpha=1.0, fit_prior=True, class_prior=None)
nb_model.fit(X_train, y_train);


In [165]:
   # save_documents = open("pickled_algos/all_pos_words.pickle","wb")
    # pickle.dump(positive_cleaned_tokens_list, save_documents)
    # save_documents.close()

In [166]:
feature_words = count_vect.get_feature_names()
n = 20 #number of top words associated with the category that we wish to see

for cat in range(len(categories)):
    print(f"\nTarget: {cat}, name: {target_names[cat]}")
    log_prob = nb_model.feature_log_prob_[cat]
    i_topn = np.argsort(log_prob)[::-1][:n]
    features_topn = [feature_words[i] for i in i_topn]
    print(f"Top {n} tokens: ", features_topn)



Target: 0, name: 1
Top 20 tokens:  ['none', 'good', 'dog', 'girl', 'meet', 'female', 'pit', 'foster', 'name', 'pounds', 'please', 'friendly', 'cats', 'boy', 'around', 'weight', 'fill', 'breed', 'spayed', 'playful']

Target: 1, name: 1
Top 20 tokens:  ['online', 'none', 'fill', 'official', 'meet', 'home', 'looking', 'dog', 'forever', 'puppy', 'name', 'lbs', 'male', 'please', 'hi', '20', 'came', 'months', 'shepherd', 'month']


In [143]:
y_predicted = nb_model.predict(X_test)
# X_test[0]

In [144]:
# y_predicted

In [145]:
from sklearn.metrics import f1_score
f1_score(y_test, y_predicted)


0.8547008547008548

In [146]:
y_pred_proba = nb_model.predict_proba(X_test)[:,1]
y_pred_proba

array([0.74267226, 0.75533722, 0.86510355, 0.90499006, 0.82601062,
       0.90632596, 0.68634114, 0.9805145 , 0.59918901, 0.78531774,
       0.71571916, 0.81663639, 0.6444431 , 0.66577122, 0.77986978,
       0.59392476, 0.72380875, 0.42476539, 0.74355175, 0.9847171 ,
       0.87370884, 0.83443539, 0.42476539, 0.9805145 , 0.82072539,
       0.55844652, 0.74944841, 0.88016423, 0.74412996, 0.78207221,
       0.65531133, 0.94498941, 0.92683811, 0.42476539, 0.70674334,
       0.63802039, 0.42476539, 0.64130663, 0.81612556, 0.7956922 ,
       0.81659505, 0.8230588 , 0.83747452, 0.97320235, 0.42476539,
       0.76310476, 0.9805145 , 0.83398418, 0.59579996, 0.71861768,
       0.66375596, 0.67124824, 0.6868805 , 0.80449991, 0.92862123,
       0.9009087 , 0.29009143, 0.69197277, 0.83177398, 0.84629935,
       0.72103772, 0.63165531, 0.7221174 , 0.83729714, 0.66642299,
       0.77101728, 0.7737819 , 0.63652364, 0.85494257, 0.63685885,
       0.60805294, 0.89994946, 0.88494304, 0.60997547, 0.68737

In [147]:
thresh=0.7
y_pred = (y_pred_proba>=thresh).astype(int)
y_pred

array([1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1])

0.7783251231527093

In [148]:
# from sklearn.pipeline import Pipeline
# nb_pipeline = Pipeline([('vect', CountVectorizer()),
#                         ('tfidf', TfidfTransformer()),
#                         ('model', MultinomialNB()),
#                         ])
# nb_pipeline.fit(twenty_train.data, twenty_train.target);

In [90]:
# twenty_test = fetch_20newsgroups(subset='test', categories=categories,
#                                      shuffle=True, random_state=42)
# docs_test = twenty_test.data
# predicted = nb_pipeline.predict(docs_test)
# accuracy = np.mean(predicted == twenty_test.target)
# print("\nThe accuracy on the test set is {0:0.3f}.".format(accuracy))

In [176]:
# next steps:
# 1. pickle and save the model 
import pickle
save_documents = open("pickled_algos/pickled_nb.pickle","wb")
pickle.dump(nb_model, save_documents)
save_documents.close()

tfidf = tfidf_transformer
save_documents = open("pickled_algos/tfidf_transformer.pickle", "wb")
pickle.dump(tfidf, save_documents)
save_documents.close()

count_vect = count_vect
save_documents = open("pickled_algos/count_vect.pickle", "wb")
pickle.dump(count_vect, save_documents)
save_documents.close()



In [183]:
# 2. test the model by 
#turn this whoel cell into a function that takes in a string
#predict_one
with open('pickled_algos/pickled_nb.pickle', 'rb') as f:
    model = pickle.load(f)

with open('pickled_algos/tfidf_transformer.pickle', 'rb') as f:
    tfidf = pickle.load(f)

with open('pickled_algos/count_vect.pickle', 'rb') as f:
    cv = pickle.load(f)

string_pred = ['this girl is a foster pit and has none of her teeth']  

cv_transformed = cv.transform(string_pred) #counts how many words
tfidf_transformed = tfidf.transform(cv_transformed)  #tf == cv . 
string_predicted = model.predict(tfidf_transformed) 
string_predicted[0]


0